<a href="https://colab.research.google.com/github/JulioLaz/chatbot_spring_01/blob/main/Chatbot_Inteligente_Alura_Sprint1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [1]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/Chatbot'

2023-09-23 16:52:44.967338: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 11.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
Mounted at /content/drive


# 2. Importar verbos

In [2]:
import pickle

archivo_pickle_verbos = "/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle"
archivo_pickle_verbos_irregulares = "/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle"

with open(archivo_pickle_verbos, "rb") as archivo:
        lista_verbos = pickle.load(archivo)

with open(archivo_pickle_verbos_irregulares, "rb") as archivo_irr:
        lista_verbos_irregulares = pickle.load(archivo_irr)


print(lista_verbos)
print('---------------------------------')
print(lista_verbos_irregulares)



['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

In [3]:
len(lista_verbos)

1747

#3. Tratamiento de datos

In [4]:
!pip install jellyfish

In [5]:
# import jellyfish

# def raiz(palabra):
#     palabra_encontrada = palabra
#     max_similitud = 0.0

#     for verbo in lista_verbos:
#         similitud = jellyfish.jaro_similarity(palabra, verbo)
#         if similitud > max_similitud:
#             max_similitud = similitud
#             palabra_encontrada = verbo

#     if max_similitud >= 0.93:
#         return palabra_encontrada, max_similitud
#     else:
#         return palabra, max_similitud


# palabra_ingresada = "caminando"
# raiz_encontrada, similitud = raiz(palabra_ingresada)
# print(f"La raíz de '{palabra_ingresada}' es '{raiz_encontrada}' con una similitud de {round(similitud,2)}")


In [6]:
# def tratamiento_texto(texto):
#   trans = str.maketrans('áéíóú','aeiou')
#   texto = texto.lower()
#   texto = texto.translate(trans)
#   # texto = re.sub(r"[^\w\s]", '', texto)
#   texto = re.sub(r"[^\w\s+\-*/]", '', texto)

#   texto = " ".join(texto.split())
#   return texto

# frase='QUÉ DÍA ES HOY,    JULIO ALBERTO?'
# texto_tratado=tratamiento_texto(frase)
# print(texto_tratado)

In [7]:
# def reemplazar_terminacion(frase):
#     terminaciones = ["es", "me", "as", "ste", "te"]
#     palabras = frase.split()

#     for i, palabra in enumerate(palabras):
#         for terminacion in terminaciones:
#             if palabra.endswith(terminacion) and len(palabra) > len(terminacion):
#                 palabras[i] = palabra[:-len(terminacion)] + "r"
#                 break

#     nueva_frase = " ".join(palabras)
#     return nueva_frase

# frase = "este me miró y asintió. Decirme algo."
# nueva_frase = reemplazar_terminacion(frase)
# print(f"Frase original: {frase}")
# print(f"Frase con las terminaciones reemplazadas: {nueva_frase}")


In [8]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
    palabra_encontrada = palabra
    max_similitud = 0.0

    for verbo in lista_verbos:
        similitud = jellyfish.jaro_similarity(palabra, verbo)
        if similitud > max_similitud:
            max_similitud = similitud
            palabra_encontrada = verbo

    if max_similitud >= 0.93:
        return palabra_encontrada, max_similitud
    else:
        return palabra, max_similitud

def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  texto = texto.translate(trans)
  # texto = re.sub(r"[^\w\s]", '', texto)
  texto = re.sub(r"[^\w\s+\-*/]", '', texto)
  texto = " ".join(texto.split())
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(frase):
    terminaciones = ["es", "me", "as", "ste", "te"]
    palabras = frase.split()

    for i, palabra in enumerate(palabras):
        for terminacion in terminaciones:
            if palabra.endswith(terminacion) and len(palabra) > len(terminacion):
                palabras[i] = palabra[:-len(terminacion)] + "r"
                break

    nueva_frase = " ".join(palabras)
    return nueva_frase

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  doc = nlp(texto)
  for t in doc:
    lemma=lista_verbos_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  return tokens

In [9]:
normalizar('Julio juega al futbol')

['julio', ('jugar', 1.0), 'futbol']

# 4. Cargar bases de documentos

In [10]:
#Importando bases de dialogo fluído
txt_folder_path = folder+'/dialogos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  estado = True
  for line in f.read().split('\n'):
    if estado:
      line_tratado = tratamiento_texto(line)
      lista_dialogos.append(line_tratado)
      lista_tipo_dialogo.append(lista_documentos[idx][:-4])
    else:
      lista_dialogos_respuesta.append(line)
    estado=not estado

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

In [11]:
len(lista_dialogos)

1152

In [12]:
df_dialogo.head()

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad
0,bueno te elogio a ti,¡Muchas gracias! Me alegro mucho de oír eso.,Agradecimiento,0,0,0
1,de acuerdo,¡Gracias!,Agradecimiento,0,0,0
2,entiendo,¡Perfecto! ¡Me alegra que hayas entendido!,Agradecimiento,0,0,0
3,estoy de acuerdo,¡Gracias!,Agradecimiento,0,0,0
4,igualmente,¡Igualmente!,Agradecimiento,0,0,0


# 5. Buscar respuesta del Chatbot

In [15]:
import pandas as pd
import jellyfish

def dialogo(pregunta_usuario, df_dialogo):
    def limpiar_pregunta(pregunta):
        pregunta_limpia = pregunta.lower()
        pregunta_limpia = ''.join(c for c in pregunta_limpia if c.isalnum() or c.isspace())
        return pregunta_limpia

    pregunta_usuario_limpia = limpiar_pregunta(pregunta_usuario)
    respuesta = ""

    for index, row in df_dialogo.iterrows():
        pregunta_dialogo = limpiar_pregunta(row['dialogo'])
        similitud = jellyfish.jaro_winkler_similarity(pregunta_usuario_limpia, pregunta_dialogo)

        if similitud >= 0.93:
            respuesta = row['respuesta']
            break
    return respuesta

pregunta_usuario = "¿Cómo te llamas?"
respuesta = dialogo(pregunta_usuario, df_dialogo)
print(respuesta)


Me llamo Madre. ¿Y tú?


#Guardar DF:

In [16]:
import pandas as pd
import os

def guardar_dataframe(dataframe):
  carpeta = "/content/drive/MyDrive/Chatbot"
  nombre_archivo = "df_dialogo.csv"

  if not os.path.exists(carpeta):
    os.makedirs(carpeta)
  archivo_csv = os.path.join(carpeta, nombre_archivo)
  dataframe.to_csv(archivo_csv, index=False)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import jellyfish

def dialogo(user_response):
    # Tratamiento de texto
    user_response = tratamiento_texto(user_response)  # Tratando el texto
    user_response = re.sub(r"[^\w\s]", '', user_response)  # Elimina signos de puntuación

    df = df_dialogo.copy()

    tfidf_vectorizer = TfidfVectorizer()

    for idx, row in df.iterrows():
        # Calcular la similitud de coseno usando TF-IDF
        tfidf_matrix = tfidf_vectorizer.fit_transform([user_response, row['dialogo']])
        cos_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

        df.at[idx, 'interseccion'] = len(set(user_response.split()) & set(row['dialogo'].split()))/len(user_response.split())
        df.at[idx, 'similarity'] = cos_sim[0][0]
        df.at[idx, 'jaro_winkler'] = jellyfish.jaro_winkler_similarity(user_response, row['dialogo'])

        # Calcular la probabilidad:
        df.at[idx, 'probabilidad'] = max(df.at[idx, 'interseccion'], df.at[idx, 'similarity'], df.at[idx, 'jaro_winkler'])

    # Ordenar el DataFrame por probabilidad y Jaro-Winkler
    df.sort_values(by=['probabilidad', 'jaro_winkler'], inplace=True, ascending=False)

    # Obtener la probabilidad máxima
    probabilidad = (df['probabilidad'].head(1).values[0])

    #Guardar df:
    guardar_dataframe(df)

    if probabilidad >= 0.93:
        print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
        respuesta = df['respuesta'].head(1).values[0]
    else:
        respuesta = ''

    return respuesta



#Cargar tu modelo entrenado aqui(recuerda siempre cargar el modelo y el vectorizer o tokenizer usado en el entrenamiento del modelo):
#Cargar el modelo entrenado
ruta_modelo = '/content/drive/MyDrive/Chatbot/'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

#Función para dialogar utilizando el modelo
def clasificacion_modelo(pregunta):
  frase = ' '.join(normalizar(pregunta))
  tokens = tokenizer_TF.encode_plus(
      frase,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  with torch.no_grad():
      outputs = Modelo_TF(input_ids, attention_mask)

  etiquetas_predichas = torch.argmax(outputs.logits, dim=1)
  etiquetas_decodificadas = etiquetas_predichas.tolist()

  diccionario = {3: 'Continuacion', 10: 'Nombre', 2: 'Contacto', 13: 'Saludos', 14: 'Sentimiento', 9: 'Identidad', 15: 'Usuario', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 0: 'Agradecimiento', 5: 'Edad', 4: 'Despedida', 11: 'Origen', 12: 'Otros', 7: 'Error', 8: 'Funcion'}
  llave_buscada = etiquetas_decodificadas[0]
  clase_encontrada = diccionario[llave_buscada]

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta

#Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = dialogo(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = clasificacion_modelo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      return 'Respuesta no encontrada'

# 6. Ejecutar Chatbot

In [19]:
pregunta='hola como te llamas?'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0
Hola, me llamo Madre, ¿y tú?
